In [15]:
# n particles. each particle has a velocity, a mass, and an acceleration
# calculate total force via a summing principle
# need to calculate v_x, v_y, v_z and a_x, a_y, a_z
# need to save positions of all particles in an velocity = [x,y,z] array
# so first create a way to take input for number of particles and randomize intial positions and velocities within a certain range
# we want to create an object of the particle. the particle will have a position (x,y,z), velocity (vx, vy, vz), acceleration (ax, ay, az) 
# calculate force from each particle by mass times acceleration on each particle in x,y, z direction. just sum up the total from the other particles to put that onto the particle
# so calculate force on each particle. create a loop and plug into newton's equations
# let's start 1000 timesteps and save all the particles. i'll save an array for each particle
# and then I will plot the total array for all particles. so i'll loop and access each of the elements of the arrays in the dth section.
# we'll stride appropriate and use a library to craft a video

In [ ]:
# N-body simulation (5 particles) 
# -----------------------------------------------------------------------------
# What this does:
# • Simulates gravitational interaction with Plummer softening (avoids blow-ups)
# • Uses velocity-Verlet (leapfrog) for stable time integration
# • Randomizes initial conditions with a seed for reproducibility
# • Prints initial masses, positions, and velocities
# • Runs a longer simulation (adjust `steps`, `stride`)
# • Exports an MP4 (fallback to GIF if ffmpeg is not available)
#
# Controls/guidance:
# • Longer movie: increase `steps` (e.g., 2000+) and keep `stride` moderate (e.g., 2–5)
# • Smaller file / faster export: increase `stride`, reduce `steps` and/or `fps`
# • Physics units are unitless (G=1); you can scale as you like

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
import pandas as pd

# -----------------------------
# 1) Physics
# -----------------------------
class Particle:
    """Minimal container for one particle's mass, position, velocity, acceleration (all 3D)."""
    def __init__(self, mass, position, velocity, acceleration=None):
        self.mass = float(mass)
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.acceleration = np.zeros(3, dtype=float) if acceleration is None else np.array(acceleration, dtype=float)

class NBodySim:
    """3D gravitational N-body (O(N^2) force sum) with velocity-Verlet integration."""
    def __init__(self, particles, G=1.0, softening=1e-2):
        self.particles = particles
        self.n = len(particles)
        self.G = float(G)
        self.eps = float(softening)
        # Dense arrays for speed (kept in sync with the list of Particle objects)
        self.m = np.array([p.mass for p in particles])         # (n,)
        self.x = np.stack([p.position for p in particles])     # (n,3)
        self.v = np.stack([p.velocity for p in particles])     # (n,3)
        self.a = np.stack([p.acceleration for p in particles]) # (n,3)

    def _compute_accelerations(self):
        """Compute accelerations a_i from all pairwise forces with Plummer softening."""
        dx = self.x[:, None, :] - self.x[None, :, :]          # (n,n,3), dx[i,j] = x_i - x_j
        r2 = np.sum(dx*dx, axis=2) + self.eps**2              # |r|^2 + eps^2
        inv_r3 = np.where(r2 > 0.0, 1.0 / np.sqrt(r2*r2*r2), 0.0)
        np.fill_diagonal(inv_r3, 0.0)                         # no self-force
        # a_i = G * sum_j m_j * dx_ij / |r_ij|^3
        a = self.G * (dx * (self.m[None, :] * inv_r3)[:, :, None]).sum(axis=1)
        a /= self.m[:, None]
        self.a = a

    def step(self, dt):
        """Velocity-Verlet (aka leapfrog) time step."""
        v_half = self.v + 0.5 * dt * self.a
        self.x = self.x + dt * v_half
        self._compute_accelerations()
        self.v = v_half + 0.5 * dt * self.a

    def simulate(self, steps=1500, dt=1e-2, stride=2):
        """
        Run the simulation and return positions over time.
        Returns: traj with shape (frames, n, 3) where frames = steps//stride + 1.
        """
        self._compute_accelerations()                 # ensure a(t) is initialized
        frames = steps // stride + 1
        traj = np.empty((frames, self.n, 3), dtype=float)
        traj[0] = self.x
        f = 1
        for t in range(steps):
            self.step(dt)
            if (t + 1) % stride == 0:
                traj[f] = self.x
                f += 1
        # Optional: sync arrays back to Particle objects
        for i, p in enumerate(self.particles):
            p.position, p.velocity, p.acceleration = self.x[i], self.v[i], self.a[i]
        return traj

# -----------------------------
# 2) Initialization
# -----------------------------
def random_system(n, pos_range=1.0, vel_range=0.12, mass_range=(0.9, 1.1), seed=2025):
    """
    Create n particles with:
      - mass ~ Uniform[mass_range]
      - position ~ Uniform[-pos_range, pos_range] per axis
      - velocity ~ Uniform[-vel_range, vel_range] per axis
    """
    rng = np.random.default_rng(seed)
    particles = []
    for _ in range(n):
        m = rng.uniform(*mass_range)
        x = rng.uniform(-pos_range, pos_range, size=3)
        v = rng.uniform(-vel_range, vel_range, size=3)
        particles.append(Particle(m, x, v))
    return particles

# -----------------------------
# 3) Configure run
# -----------------------------
n = 5
steps = 1500     # increase for a longer movie (e.g., 3000+)
dt = 1e-2
stride = 2       # save every 2 steps (raises/decreases frame count)
seed = 2025
pos_range = 1.0
vel_range = 0.12
mass_range = (0.9, 1.1)
G = 5.0
softening = 1e-2

# Build system and print initial values
particles = random_system(n, pos_range=pos_range, vel_range=vel_range,
                          mass_range=mass_range, seed=seed)

init_df = pd.DataFrame({
    "mass": [p.mass for p in particles],
    "x": [p.position[0] for p in particles],
    "y": [p.position[1] for p in particles],
    "z": [p.position[2] for p in particles],
    "vx": [p.velocity[0] for p in particles],
    "vy": [p.velocity[1] for p in particles],
    "vz": [p.velocity[2] for p in particles],
})
print("=== Initial Values for 5 particles ===")
print(init_df.to_string(index=False))

# -----------------------------
# 4) Run simulation
# -----------------------------
sim = NBodySim(particles, G=G, softening=softening)
traj = sim.simulate(steps=steps, dt=dt, stride=stride)  # shape (frames, n, 3)

# -----------------------------
# 5) Animate & Save
# -----------------------------
frames = traj.shape[0]
xyz_max = float(np.abs(traj).max() * 1.1) if frames > 0 else 1.0

fig = plt.figure(figsize=(5.2, 5.2))    # modest size for faster encoding
ax = fig.add_subplot(projection='3d')
scat = ax.scatter(traj[0,:,0], traj[0,:,1], traj[0,:,2], s=50)
ax.set_xlim(-xyz_max, xyz_max); ax.set_ylim(-xyz_max, xyz_max); ax.set_zlim(-xyz_max, xyz_max)
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
ax.set_title("N-body (5 particles)")

def update(i):
    scat._offsets3d = (traj[i,:,0], traj[i,:,1], traj[i,:,2])
    ax.set_title(f"N-body (5) — frame {i}/{frames-1}")
    return scat,

anim = FuncAnimation(fig, update, frames=frames, interval=40, blit=False)

# Prefer MP4 (ffmpeg), fallback to GIF
out_mp4 = "nbody_5_particles_long.mp4"
out_gif = "nbody_5_particles_long.gif"
try:
    writer = FFMpegWriter(fps=20, metadata={'artist': 'N-body Notebook'})
    anim.save(out_mp4, writer=writer, dpi=110)
    print(f"Saved MP4: {out_mp4}")
except Exception as e:
    writer = PillowWriter(fps=12)
    anim.save(out_gif, writer=writer, dpi=95)
    print(f"FFmpeg unavailable, saved GIF instead: {out_gif}")

plt.close(fig)

=== Initial Values for 5 particles ===
    mass         x         y         z        vx        vy        vz
1.098892 -0.235981  0.654296  0.674511  0.114194 -0.101466 -0.043810
1.083911  0.351739 -0.428350 -0.220914 -0.064747 -0.079929 -0.083175
1.094818 -0.152208 -0.873712 -0.016827 -0.050798 -0.069212 -0.070216
1.015804  0.181701  0.776129 -0.780334 -0.000887 -0.050368 -0.003396
1.089704  0.853018  0.520201  0.564052 -0.053302  0.004229  0.073100
FFmpeg unavailable, saved GIF instead: nbody_5_particles_long.gif
